In [40]:
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
#import emoji 
from collections import Counter 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [41]:
def date_time(s):
    pattern = '^([0-9]+(\/)([0-9]+)(\/)[0-9]+, ([0-9]+):([0-9]+)\s(PM|AM|am|pm) - )'
    result = re.match(pattern, s)
    if result:
        return True 
    return False

def contact(s):
    s=s.split(":")
    if len(s) == 2:
        return True 
    return False

def getmsg(line):
    splitline = line.split(' - ')
    date, time = splitline[0].split(', ')
    msg = " ".join(splitline[1:])
   
    if contact(msg):
        split_msg = msg.split(': ')
        author = split_msg[0]
        msg = " ".join(split_msg[1:])
    else:
        author = None
    return date, time, author, msg


<>:2: SyntaxWarning: invalid escape sequence '\/'
<>:2: SyntaxWarning: invalid escape sequence '\/'
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/4058073619.py:2: SyntaxWarning: invalid escape sequence '\/'
  pattern = '^([0-9]+(\/)([0-9]+)(\/)[0-9]+, ([0-9]+):([0-9]+)\s(PM|AM|am|pm) - )'


In [42]:
data = []
conversation = 'chats/WhatsApp Chat with Vishal Sir.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    msgBuffer = []
    date, time, author=None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()

        if date_time(line):
            if len(msgBuffer) > 0 :
                data.append([date, time, author, " ".join(msgBuffer)])
            msgBuffer.clear()
            date, time, author, msg = getmsg(line)
            msgBuffer.append(msg)
        else:
            msgBuffer.append(line)

In [43]:
msgBuffer

['It’s ok sir', 'Any time']

In [44]:
data = pd.DataFrame(data, columns=["Date", "Time", "Contact", "Message"])
data['Date'] = pd.to_datetime(data['Date'])
data = data.dropna()
data

/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/337529315.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date'] = pd.to_datetime(data['Date'])


,Date,Time,Contact,Message
0,2021-03-31,7:44 PM,Sai Nithin Kadarla,Sir this too
1,2021-03-31,7:47 PM,Sai Nithin Kadarla,<Media omitted>
2,2021-04-07,5:59 PM,Sai Nithin Kadarla,You deleted this message
3,2021-04-07,5:59 PM,Sai Nithin Kadarla,<Media omitted>
4,2021-04-07,5:59 PM,Sai Nithin Kadarla,sir i did it
...,...,...,...,...
129,2024-05-30,11:56 AM,Sai Nithin Kadarla,Ok sir
130,2024-05-30,11:58 AM,Sai Nithin Kadarla,<Media omitted>
131,2024-05-30,11:59 AM,Sai Nithin Kadarla,<Media omitted>
132,2024-05-30,11:59 AM,Sai Nithin Kadarla,Ou certificate sir


In [45]:
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
              "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
              "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
              "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do",
              "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
              "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
              "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
              "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
              "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
              "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [46]:

# condition_1 = data["Message"].str.contains(" deleted this message")
# condition_2 = data["Message"] == "<Media omitted>"

# # Drop rows based on conditions
# data_dropped = data.drop(data[condition_2])
# data_dropped

# Print DataFrame after dropping rows based on conditions
# print("\nDataFrame after dropping rows based on conditions:")


In [47]:
data["Message"] = data['Message'][data["Message"] != "<Media omitted>"]
print(data)
data.dropna(axis=0, inplace=True)
print(data)

string_to_match = " deleted this message"

# Drop rows based on string match
data = data[~data["Message"].str.contains(string_to_match, case=False)]
data


          Date      Time             Contact                   Message
0   2021-03-31   7:44 PM  Sai Nithin Kadarla              Sir this too
1   2021-03-31   7:47 PM  Sai Nithin Kadarla                       NaN
2   2021-04-07   5:59 PM  Sai Nithin Kadarla  You deleted this message
3   2021-04-07   5:59 PM  Sai Nithin Kadarla                       NaN
4   2021-04-07   5:59 PM  Sai Nithin Kadarla              sir i did it
..         ...       ...                 ...                       ...
129 2024-05-30  11:56 AM  Sai Nithin Kadarla                    Ok sir
130 2024-05-30  11:58 AM  Sai Nithin Kadarla                       NaN
131 2024-05-30  11:59 AM  Sai Nithin Kadarla                       NaN
132 2024-05-30  11:59 AM  Sai Nithin Kadarla        Ou certificate sir
133 2024-05-30  12:02 PM          Vishal Sir         Thank u ...nithin

[128 rows x 4 columns]
          Date      Time             Contact  \
0   2021-03-31   7:44 PM  Sai Nithin Kadarla   
2   2021-04-07   5:59 PM  Sa

,Date,Time,Contact,Message
0,2021-03-31,7:44 PM,Sai Nithin Kadarla,Sir this too
4,2021-04-07,5:59 PM,Sai Nithin Kadarla,sir i did it
5,2021-05-05,9:37 PM,Sai Nithin Kadarla,"Sir , is online classes going on ?"
6,2021-06-30,2:35 PM,Sai Nithin Kadarla,Thank you sir
14,2021-12-03,4:35 PM,Sai Nithin Kadarla,Ok sir
...,...,...,...,...
127,2024-05-30,11:56 AM,Vishal Sir,S
128,2024-05-30,11:56 AM,Vishal Sir,Send ou also
129,2024-05-30,11:56 AM,Sai Nithin Kadarla,Ok sir
132,2024-05-30,11:59 AM,Sai Nithin Kadarla,Ou certificate sir


In [48]:
data['Cleaned_message'] = data["Message"].apply(lambda x: x.lower().translate(str.maketrans(' ', " ", string.punctuation)))

data['Tokenized_words'] = data["Cleaned_message"].apply(lambda y: [word for word in word_tokenize(y) if word not in STOPWORDS and word not in stop_words] )

/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/2113895370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Cleaned_message'] = data["Message"].apply(lambda x: x.lower().translate(str.maketrans(' ', " ", string.punctuation)))
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/2113895370.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tokenized_words'] = data["Cleaned_message"].apply(lambda y: [word for word in word_tokenize(y) if word not in STOPWORDS and word

In [49]:
data.head(50)

,Date,Time,Contact,Message,Cleaned_message,Tokenized_words
0,2021-03-31,7:44 PM,Sai Nithin Kadarla,Sir this too,sir this too,[sir]
4,2021-04-07,5:59 PM,Sai Nithin Kadarla,sir i did it,sir i did it,[sir]
5,2021-05-05,9:37 PM,Sai Nithin Kadarla,"Sir , is online classes going on ?",sir is online classes going on,"[sir, online, classes, going]"
6,2021-06-30,2:35 PM,Sai Nithin Kadarla,Thank you sir,thank you sir,"[thank, sir]"
14,2021-12-03,4:35 PM,Sai Nithin Kadarla,Ok sir,ok sir,"[ok, sir]"
16,2021-12-27,7:17 PM,Vishal Sir,Send this yo srujan,send this yo srujan,"[send, yo, srujan]"
17,2021-12-27,7:20 PM,Sai Nithin Kadarla,Ok sir,ok sir,"[ok, sir]"
18,2022-01-12,11:45 AM,Sai Nithin Kadarla,"Hi sir, good morning",hi sir good morning,"[hi, sir, good, morning]"
19,2022-01-12,11:45 AM,Sai Nithin Kadarla,Our documentation work is completed,our documentation work is completed,"[documentation, work, completed]"
20,2022-01-12,11:45 AM,Sai Nithin Kadarla,can I send you sir,can i send you sir,"[send, sir]"


In [50]:
lemmatizer = WordNetLemmatizer()
data["Lemmatized"] = [[lemmatizer.lemmatize(token) for token in token_list] for token_list in data["Tokenized_words"]]


/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/964848139.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Lemmatized"] = [[lemmatizer.lemmatize(token) for token in token_list] for token_list in data["Tokenized_words"]]


In [51]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data['Positive'] = [sentiments.polarity_scores(i)['pos'] for i in data["Message"]]
data['Negative'] = [sentiments.polarity_scores(i)['neg'] for i in data["Message"]]
data['Neutral'] = [sentiments.polarity_scores(i)['neu'] for i in data["Message"]]

/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/1503036832.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Positive'] = [sentiments.polarity_scores(i)['pos'] for i in data["Message"]]
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/1503036832.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Negative'] = [sentiments.polarity_scores(i)['neg'] for i in data["Message"]]
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/1503036832.py:5: SettingWith

In [54]:
data["Positive"] = data["Positive"].apply(lambda x: np.ceil(x) if x - np.floor(x) >= 0.5 else np.floor(x))
data["Negative"] = data["Negative"].apply(lambda x: np.ceil(x) if x - np.floor(x) >= 0.5 else np.floor(x))
data["Neutral"] = data["Neutral"].apply(lambda x: np.ceil(x) if x - np.floor(x) >= 0.5 else np.floor(x))


/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/6469762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Positive"] = data["Positive"].apply(lambda x: np.ceil(x) if x - np.floor(x) >= 0.5 else np.floor(x))
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_1206/6469762.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Negative"] = data["Negative"].apply(lambda x: np.ceil(x) if x - np.floor(x) >= 0.5 else np.floor(x))
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/i

In [55]:
data

,Date,Time,Contact,Message,Cleaned_message,Tokenized_words,Lemmatized,Positive,Negative,Neutral
0,2021-03-31,7:44 PM,Sai Nithin Kadarla,Sir this too,sir this too,[sir],[sir],0.0,0.0,1.0
4,2021-04-07,5:59 PM,Sai Nithin Kadarla,sir i did it,sir i did it,[sir],[sir],0.0,0.0,1.0
5,2021-05-05,9:37 PM,Sai Nithin Kadarla,"Sir , is online classes going on ?",sir is online classes going on,"[sir, online, classes, going]","[sir, online, class, going]",0.0,0.0,1.0
6,2021-06-30,2:35 PM,Sai Nithin Kadarla,Thank you sir,thank you sir,"[thank, sir]","[thank, sir]",1.0,0.0,0.0
14,2021-12-03,4:35 PM,Sai Nithin Kadarla,Ok sir,ok sir,"[ok, sir]","[ok, sir]",1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
127,2024-05-30,11:56 AM,Vishal Sir,S,s,[],[],0.0,0.0,0.0
128,2024-05-30,11:56 AM,Vishal Sir,Send ou also,send ou also,"[send, ou]","[send, ou]",0.0,0.0,1.0
129,2024-05-30,11:56 AM,Sai Nithin Kadarla,Ok sir,ok sir,"[ok, sir]","[ok, sir]",1.0,0.0,0.0
132,2024-05-30,11:59 AM,Sai Nithin Kadarla,Ou certificate sir,ou certificate sir,"[ou, certificate, sir]","[ou, certificate, sir]",0.0,0.0,1.0


In [57]:
pos = sum(data['Positive'])
neg = sum(data['Negative'])
neu = sum(data['Neutral'])

def score(a,b,c):
    if a>b and a>c:
        print('postive')
    elif b>a and b>c:
        print('negative')
    else:
        print("neutral")

score(pos, neg, neu)

pos, neg, neu

neutral


(23.0, 0.0, 59.0)

In [67]:
combined_words = [word for sublist in data['Lemmatized'] for word in sublist]

wordcounter = Counter(combined_words)
common_words = wordcounter.most_common(10)
print(common_words)

[('sir', 51), ('ok', 13), ('thank', 8), ('send', 7), ('please', 7), ('certificate', 6), ('one', 5), ('name', 5), ('call', 5), ('good', 4)]


In [70]:
from transformers import pipeline
senti_pipe = pipeline('SentimentIntensityAnalyzer')
sentiments = senti_pipe(data["Message"])

KeyError: "Unknown task SentimentIntensityAnalyzer, available tasks are ['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

['sir',
 'sir',
 'sir',
 'online',
 'class',
 'going',
 'thank',
 'sir',
 'ok',
 'sir',
 'send',
 'yo',
 'srujan',
 'ok',
 'sir',
 'hi',
 'sir',
 'good',
 'morning',
 'documentation',
 'work',
 'completed',
 'send',
 'sir',
 'ok',
 'sir',
 'cme',
 'sir',
 'ece',
 'one',
 'project',
 'name',
 'traffic',
 'signal',
 'controller',
 'smoke',
 'detector',
 'vacuum',
 'cleaner',
 'using',
 'rasberry',
 'pi',
 'sir',
 'call',
 'set',
 'doc',
 'a4',
 'yeah',
 'sir',
 'sir',
 'call',
 'sir',
 'floor',
 'cleaner',
 'developed',
 'using',
 'pico',
 'project',
 'name',
 'rest',
 'document',
 'ok',
 'na',
 'sir',
 'smart',
 'automative',
 'module',
 'using',
 'raspberry',
 'pi',
 'microcontrollers',
 'smart',
 'automative',
 'module',
 'using',
 'rpi',
 'microcontrollers',
 'sir',
 'please',
 'suggest',
 'one',
 'title',
 'sir',
 'call',
 'sir',
 'github',
 'repository',
 'ok',
 'hello',
 'sir',
 'want',
 'call',
 'sir',
 'morning',
 '10',
 'ki',
 'ok',
 'kada',
 'sir',
 'sorry',
 'sir',
 'time',
 